<a href="https://colab.research.google.com/github/KaihangZhao/DL_Notebook_Warehouse/blob/main/Recommendation_System/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [ ]:
cuda = torch.device('cuda')

In [ ]:
item = pd.read_csv('item_feature.csv')
train = pd.read_csv('training.csv')
df = train.merge(item, on = 'item_id', how = 'left')
df['label'] =1
train = df.copy()

In [ ]:
u = np.random.randint(low=0.0, high=df.user_id.max(), size=int(len(df)*1))
i = np.random.randint(low=0.0, high=df.item_id.max(), size=int(len(df)*1))
p = (df.context_feature_id.value_counts()/len(df)).sort_index().values
p[p.argmin()]+=1-p.sum()
c = np.random.choice(4, len(df)*10, p=p)

In [ ]:
sample= pd.concat([pd.Series(u),pd.Series(i)], axis =1).\
rename(columns={0:'user_id', 1:'item_id'})
sample = sample.merge(item, on = 'item_id', how = 'left')
sample['label'] = 0
df = pd.concat([df,sample])
df = df.drop_duplicates(subset=['user_id','item_id']).reset_index(drop = True)
df = df[df['label']==0]
df = pd.concat([train, df]).reset_index(drop=True)

In [ ]:
pos = df[df.label ==1].reset_index(drop = True)
neg = df[df.label ==0].reset_index(drop = True)

In [ ]:
def data_sample(pos, neg):

  msk = np.random.rand(len(pos)) < 0.8
  train_pos = pos[msk].reset_index(drop = True)
  val_pos = pos[~msk].reset_index(drop = True)

  msk = np.random.rand(len(neg)) < 0.8
  train_neg = neg[msk].sample(frac = len(neg)/len(pos)).reset_index(drop = True)
  val_neg = neg[~msk].sample(frac = len(neg)/len(pos)).reset_index(drop = True)

  train = pd.concat([train_pos, train_neg]).sample(frac=1).reset_index(drop = True)
  val = pd.concat([val_pos, val_neg]).sample(frac=1).reset_index(drop = True)
  
  return train, val

In [ ]:
train, val = data_sample(pos, neg)

In [ ]:
class NN(nn.Module):
    def __init__(self, num_users, num_items, num_item_feature, emb_user_size=20, emb_item_size=20, emb_item_feature_size=20, seed=23):
        super(MF, self).__init__()
        torch.manual_seed(seed)
        self.user_emb = nn.Embedding(num_users, emb_user_size)
        self.item_emb = nn.Embedding(num_items, emb_item_size)
        self.item_feature_emb = nn.Embedding(num_item_feature, emb_item_feature_size)
        # init 
        self.item_feature_emb.weight.data.uniform_(0,0.05)
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.classifier = nn.Sigmoid()
        self.linear1 = nn.Linear(emb_user_size+emb_item_size+emb_item_feature_size, 2)
        self.nonlin = nn.LeakyReLU()
        self.linear3 = nn.Linear(2,1)
        
    def forward(self, u, v, c):
        U = self.user_emb(u)
        V = self.item_emb(v)
        C = self.item_feature_emb(c)
        ensemble = torch.cat((U,V,C),dim=1)
        pred = self.linear1(ensemble)
        pred = self.nonlin(pred)
        pred = self.linear3(pred)
        self.U = U
        self.V = V
        self.C = C
        return self.classifier(pred.squeeze())
        ### END SOLUTION
  
    def __getitem__(self,idx):

        return self.U[idx], self.V[idx], self.C[idx]

In [ ]:
def train_one_epoch(model, train_df, optimizer):
  tensor_x_tr = torch.LongTensor(np.array(train_df[['user_id', 'item_id', 'item_feature_id']])) # transform to torch tensor
  tensor_y_tr = torch.Tensor(train_df['label'])
  train_ds = TensorDataset(tensor_x_tr,tensor_y_tr)
  train_dl = DataLoader(train_ds, batch_size=50000, shuffle=True)
  """ Trains the model for one epoch"""
  ### BEGIN SOLUTION
  losses = []
  acc=[]
  for x, y in train_dl:
      model.train()
      y = y.cuda()
      u = torch.LongTensor(x[:,0]).cuda()
      v = torch.LongTensor(x[:,1]).cuda()
      c = torch.LongTensor(x[:,2]).cuda()
      y_hat = model(u,v,c)
      output = torch.as_tensor(y_hat > 0.5, dtype = torch.int8)
      loss = F.binary_cross_entropy(y_hat, y.float())
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      losses.append(loss.item())
      acc.append(accuracy_score(output.cpu().detach().numpy(),y.cpu().detach().numpy()))
      
  train_loss = np.mean(losses)
  train_acc = np.mean(acc)

    ### END SOLUTION
  return train_loss, train_acc

def valid_metrics(model, valid_df):
    """Computes validation loss and accuracy"""
    model.eval()
    ### BEGIN SOLUTION
    u = torch.LongTensor(valid_df.user_id.values).cuda()
    v = torch.LongTensor(valid_df.item_id.values).cuda()
    c = torch.LongTensor(valid_df.item_feature_id.values).cuda()
    y = torch.FloatTensor(valid_df.label.values).cuda()
    y_hat = model(u,v,c)
    valid_loss = F.binary_cross_entropy(y_hat, y)
    output = torch.as_tensor(y_hat > 0.5, dtype = torch.int8)
    auc = roc_auc_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
    valid_acc = accuracy_score(output.cpu().detach().numpy(),y.cpu().detach().numpy())
    ### END SOLUTION
    return valid_loss.item(), valid_acc, auc

def training(model, pos, neg, epochs=10, lr=0.01, wd=0.0):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    train, val = data_sample(pos, neg)
    for i in range(epochs):
        train_loss, train_acc = train_one_epoch(model, train, optimizer)
        valid_loss, valid_acc, auc = valid_metrics(model, val) 
        if valid_loss<0.31: 
          print("train loss %.3f train acc %.3f valid loss %.3f valid acc %.3f roc auc acc %.3f" % (train_loss,train_acc,valid_loss, valid_acc, auc)) 
          break
        if i%2 == 0: 
          train, val = data_sample(pos, neg)
        if i%5 == 0:
          print("train loss %.3f train acc %.3f valid loss %.3f valid acc %.3f roc auc acc %.3f" % (train_loss,train_acc,valid_loss, valid_acc, auc)) 
        


In [ ]:
model = MF(df.user_id.max()+1, df.item_id.max()+1, df.item_feature_id.max()+1,
           emb_user_size=32, emb_item_size=24, emb_item_feature_size=8).cuda()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
training(model, pos, neg, epochs=51, lr=0.0005, wd=1e-6)

train loss 0.749 train acc 0.500 valid loss 0.746 valid acc 0.500 roc auc acc 0.886
train loss 0.647 train acc 0.500 valid loss 0.627 valid acc 0.500 roc auc acc 0.935
train loss 0.429 train acc 0.875 valid loss 0.414 valid acc 0.874 roc auc acc 0.945
train loss 0.320 train acc 0.879 valid loss 0.314 valid acc 0.880 roc auc acc 0.954
train loss 0.305 train acc 0.883 valid loss 0.307 valid acc 0.881 roc auc acc 0.954


In [ ]:
# stop, but continue training until a second stop
training(model, pos, neg, epochs=51, lr=0.0001, wd=1e-6)

train loss 0.300 train acc 0.884 valid loss 0.300 valid acc 0.883 roc auc acc 0.956


In [ ]:
test = pd.read_csv('test_kaggle.csv')
test = test.merge(item, on = 'item_id', how = 'left')
u = torch.LongTensor(test.user_id.values).cuda()
v = torch.LongTensor(test.item_id.values).cuda()
c = torch.LongTensor(test.item_feature_id.values).cuda()
y_hat = model(u,v,c)
y_hat

tensor([0.5350, 0.2149, 0.8678,  ..., 0.9268, 0.9268, 0.1153], device='cuda:0',
       grad_fn=<SigmoidBackward0>)

In [ ]:
np.mean(y_hat.cpu().detach().numpy())

0.51798266

In [ ]:
prob = pd.Series(y_hat.cpu().detach().numpy()).reset_index().rename(columns = {'index':'id',0:'rating'})
sum(prob.rating>0.5)/len(prob)

0.5012703698362547

In [ ]:
prob.to_csv('trial63.csv',index = False)